**Привет!**

Меня зовут Артемьева Саша, я буду проверять твой проект. Приятно познакомиться, хоть и заочно:)
    
Предлагаю общаться на «ты» :) Но если это не удобно - дай знать, и мы перейдем на "вы". 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе стать аналитиком данных. Ты уже проделал большую работу над проектом, но давай сделаем его еще лучше. Ниже ты найдешь мои комментарии - пожалуйста, не перемещай, не изменяй и не удаляй их. Я буду использовать цветовую разметку:

<div class="alert alert-danger">
<b>❌ Комментарий ревьюера:</b> Так выделены самые важные замечания. Без их отработки проект не будет принят. </div>

<div class="alert alert-warning">
<b>⚠️ Комментарий ревьюера:</b> Так выделены небольшие замечания. Одно-два таких замечания в проекте допустимы, но если их будет больше - тебе будет необходимо внести исправления. Это как тестовое задание при приеме на работу: очень много мелких ошибок могут стать причиной отказа кандидату. 

</div>

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера:</b> Так я выделяю все остальные комментарии.</div>

Давай работать над проектом в диалоге: если ты что-то меняешь в проекте или отвечаешь на мои комменатри — пиши об этом. Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>Комментарий студента:</b> Например, вот так.</div>

Всё это поможет выполнить повторную проверку твоего проекта оперативнее. 


# Проект - 2: Исследование надёжности заёмщиков

## Введение

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - проверить 4 гипотезы.
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Описание данных**

В таблице 12 столбцов:
1. `children` — количество детей в семье
2. `days_employed` — общий трудовой стаж в днях
3. `dob_years` — возраст клиента в годах
4. `education` — уровень образования клиента
5. `education_id` — идентификатор уровня образования
6. `family_status` — семейное положение
7. `family_status_id` — идентификатор семейного положения
8. `gender` — пол клиента
9. `income_type` — тип занятости
10. `debt` — имел ли задолженность по возврату кредитов (да / нет)
11. `total_income` — ежемесячный доход
12. `purpose` — цель получения кредита

Каждая строчка в дата сете - запись о клиенте. Здесь есть как категориальные данные (`education`, `income_type`), так и количественные переменные (`total_income`, `children`). Похоже, что данных достаточно для проверки поставленных гипотез.

Дальше в анализе мы проверим есть ли какие-то артефакты в данных, которые требуют внимания. Но хотя бы все названия столбцов уже записаны в `snake_case` формате.

## Предобработка данных

Для начала проверим какие данные нам достались. Нас интересуют:

1. Артефакты в данных
2. Отсутствующие (None) значения в стобцах
3. Повторяющиеся строки, нестандратные названия категорий и типы данных в колонках
4. Словари (lookup tables)
5. Классификация `total_income`

Пойдем по порядку.

In [1]:
# Подключаем библиотеку и открываем дата сет
import pandas as pd
import numpy as np
from IPython.display import display

# Я привык сохранять исходный датасет, чтобы можно было проследить какие преобразования были совершены.
raw_df = pd.read_csv( '/datasets/data.csv' )
df = raw_df.copy()

In [2]:
# Выведем на экран информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Первое, что бросается в глаза - категориальные колонки имеют формат `object` - текстовые поля. Большинство количественных колонок имеют формат `float`, кроме `children` и `dob_years` - эти колонки имеют формат `int` (что логично, потому что дробных детей не бывает).

Второе - колонки имеют разное количество строк (`days_employed` и `total_income` примерно на 2000 строк короче, чем остальные). Это говорит о том, что для некоторых записей (строк) эти значения отсутствуют.

Третье - похоже, что колонки `education_id` и `family_status_id` (в формате `int`) содержат идентификаторы (ключи) для различных статусов в колонках `education` и `family_status`.

In [3]:
# Выведем на экран первые 10 строчек
display( df.head( 10 ) )

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Уже по первым 10 строчкам понятно, что данные надо будет обработать перед анализом. К примеру, в колонке `education` данные не стандартизованы: значение может быть "СРЕДНЕЕ", а может быть "Среднее". В колонке `days_employed` у нас также встречаются странные значения: либо отрицательные, либо больше 300К.

### 1 - Артефакты в данных

Прежде чем мы начнем какие-либо операции с данными (например, замену пустых значений средними), проверим и исправим возможные артефакты.

In [4]:
# Выберем 'количественные колонки'
columns_values = ['children',
                  'days_employed',
                  'dob_years',
                  'total_income']

# Выберем 'категориальные колонки'
columns_category = ['education',
                    'education_id',
                    'family_status',
                    'family_status_id',
                    'gender',
                    'income_type',
                    'debt',
                    'purpose' ]

# Проверим количественные колонки
round( df[ columns_values ].describe(), 1 )

,children,days_employed,dob_years,total_income
count,21525.0,19351.0,21525.0,19351.0
mean,0.5,63046.5,43.3,167422.3
std,1.4,140827.3,12.6,102971.6
min,-1.0,-18388.9,0.0,20667.3
25%,0.0,-2747.4,33.0,103053.2
50%,0.0,-1203.4,42.0,145017.9
75%,1.0,-291.1,53.0,203435.1
max,20.0,401755.4,75.0,2265604.0


Нам не повезло: нас ждут дети в количестве -1 и 20, заявители из ясельной группы (возраст равен нулю), а также отрицательный (и тысячелетний(!)) стаж работы.

В нормальной ситуации, у меня было бы много вопросов к владельцу этих данных. Но у нас образовательное упражнение, поэтому я исправлю эти артефакты исходя из нескольких предположений:

1. `children = -1` и `children = 20` - вероятно, какая-то логика исправления ошибок в системе. Видимо, при ошибке ввода в базу записывается -1. А 20 - ограничение сверху (это также подтверждается отсутствием значений между 5 и 20). Заменим их средним значением для соответствующей группы `income_type`. Так, у пенсионера, скорей всего, будет несколько детей, а у студента - ни одного. Это предположение натянуто для `children = 20`, но, учитывая сжатые сроки, других у меня нет. :(

2. Сделаем то же самое для `dob_years`. Вряд ли мы увидим 60-летних студентов, или 20-летних пенсионеров. 

3. `days_employed` немного сложнее. У нас здесь две проблемы: отрицательные значения и тысячелетия. Отрицательные значения можно исправить функцией `abs()` (возможно, ошибки импорта данных). Я думал, что тысячелетия - это, стаж работы в часах, а не днях. После трех часов раскопок, я все равно получил `days_employed` >> `dob_years`...Т.к эта колонка не будет использована в проверке гипотез, то мы ее просто убьем.

Выполним эти приседания.

In [5]:
# 1. Заменим подозрительное количество детей на NaN
df.children = df.children.replace( { -1:np.nan, 20:np.nan } )

# Заменим NaN на среднее количество детей в каждой категории income_type
df.children = df.children.fillna( df.groupby( 'income_type' ).children.transform( 'mean' ) )

# Убедимся, что все дети целочисленные. Мы сломали тип данных в этой колонке - починим в следующей секции
df.children = round( df.children )

# 2. Заменяем неродившихся в колонке dob_years на NaN
df.dob_years = df.dob_years.replace( 0, np.nan )

# Заменим NaN на средний возраст в каждой категории income_type
df.dob_years = df.dob_years.fillna( df.groupby( 'income_type' ).dob_years.transform( 'mean' ) )

# 3. Сделаем days_employed положительными
df.drop( 'days_employed', inplace=True, axis=1 )
columns_values.remove( 'days_employed' )

# Проверим успех
round( df[ columns_values ].describe(), 1 )

,children,dob_years,total_income
count,21525.0,21525.0,19351.0
mean,0.5,43.5,167422.3
std,0.8,12.2,102971.6
min,0.0,19.0,20667.3
25%,0.0,34.0,103053.2
50%,0.0,43.0,145017.9
75%,1.0,53.0,203435.1
max,5.0,75.0,2265604.0


<div class="alert alert-success">
<b>✔️ Комментарий ревьюера:</b>  Отличная замена аномалий! В целом, можно было посчитать их долю и если она в пределах 5%, смло от нее избавиться

Ура! Эти значения теперь выглядит достаточно прилично. Посмотрим на вторую часть этого упражнения.

In [6]:
# Проверим уникальные значения в 'категориальные' колонках
for column in columns_category:
    print( 'Название колонки:', column )
    print( 'Список уникальных значений:' )
    display( raw_df[ column ].value_counts() )
    print( '-' * 50 )

Название колонки: education
Список уникальных значений:


среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

--------------------------------------------------
Название колонки: education_id
Список уникальных значений:


1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

--------------------------------------------------
Название колонки: family_status
Список уникальных значений:


...

--------------------------------------------------
Название колонки: debt
Список уникальных значений:


0    19784
1     1741
Name: debt, dtype: int64

--------------------------------------------------
Название колонки: purpose
Список уникальных значений:


свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

--------------------------------------------------


Нам повезло: приключения нас ждут только в колонках `education` и `family_status` (разный регистр), `gender` (имеется странное значение `XNA`) и `purpose` (похожие по смыслу категории 'приобретение автомобиля' и 'покупка автомобиля'). Эти проблемы (кроме `XNA`) мы устраним в следующих секциях.

In [7]:
# Уберем XNA из датасета
df = df[ df.gender != 'XNA' ]

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера:</b>  Молодец, что нашел этот выброс!

### 2 - Отсутствующие значения в стобцах
Поехали дальше.

In [8]:
# Посчитаем количество пропусков в данных
df.isna().sum()

children               0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как мы и предполагали - у нас есть пропуски в колонке `total_income`. Посмотрим какую долю от общего количества строк они составляют.

Количество пустых значений также равно 2174 - значит в одной строчке значение пропущено в двух столбцах. Тогда можем работать с пустыми значениями построчно.

Теперь проверим какую долю пустые строки занимают во всем датасете.

In [9]:
# Поделим количество пропущенных строк в total_income на общее количесто строк (возьмем children как колонку со всеми значениями)
print( 'Доля пустых строк в датасете:', round( df.total_income.isna().sum() / df.children.count() * 100, 1), '%'  )
display( df[ df.total_income.isna() ].head() )

Доля пустых строк в датасете: 10.1 %


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0.0,65.0,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0.0,41.0,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0.0,63.0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0.0,50.0,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0.0,54.0,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Нам опять повезло: количество пропущенных значений относительно мало. Похоже, что они все случайны: вряд ли у нас был мужчина-госслужащий даже со средним образованием без опыта работы и без какого-либо заработка (см. строку 26). Поэтому, мы можем заменить эти значения средними в рамках одного типа занятости: такой подход несильно исказит общую картину, учитывая, что у нас может быть большой разбег в значениях дохода для разных групп.

In [10]:
# Перезапишем колонку totol_income. Теперь вместо пустых значений будут стоять средние значения для каждой категории income_type
df.total_income = df.total_income.fillna( df.groupby('income_type').total_income.transform( 'mean' ) )

### 3 - Повторяющиеся строки, нестандратные названия категорий и типы данных в колонках

Пора посмотреть на возможные повторы. Т.к. у нас нету уникальных идентификаторов записи в датасете, то я подойду к проблеме напрямую: если есть строчки, где все значения одинаковые - будем считать это дубликатом.

В новом задании предлагается сначала стандартизировать регистр и типы данных. Я предлагаю сначала удалить дубликаты, а потом уже приступить к стандартизации: во-первых, `total_income` имеет много знаков после запятой (может, значение было переведено из одной валюты в другую) - этим можно повысить "уникальность" строки; во-вторых, если предположить, что разный регистр в `income_type` - это следствие ручного ввода данных человеком, то это тоже поможет избежать удаления лишних строк.

In [11]:
# Посчитаем количество повторяющихся строк (без учета первой встретившейся строки)
print( 'Количество повторяющихся строк:', df.duplicated().sum() )

# Посмотрим на виноватых
display( df[ df.duplicated() ].sort_values( by=['income_type', 'dob_years'] ).head() )

Количество повторяющихся строк: 54


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
18349,1.0,30.0,высшее,0,женат / замужем,0,F,госслужащий,0,170898.309923,покупка жилья для семьи
14432,2.0,36.0,высшее,0,женат / замужем,0,F,госслужащий,0,170898.309923,получение образования
13878,1.0,31.0,среднее,1,женат / замужем,0,F,компаньон,0,202417.136353,покупка жилья
19387,0.0,38.0,высшее,0,гражданский брак,1,F,компаньон,0,202417.136353,на проведение свадьбы
10697,0.0,40.0,среднее,1,гражданский брак,1,F,компаньон,0,202417.136353,сыграть свадьбу


Похоже, все повторяющиеся строки случайны: в этих заявках не были указаны значения `total_income` (поэтому они одинаковые - мы их заменили средними). Так как мы не значем что такое NaN в контексте этих данных (значение не указано, потому что пропустили или потому что оно равно нулю), то простым решением будет вырезать эти строки из датасета.

In [12]:
# Уберем повторяющиеся строки - сделаем это до исправления регистра
df = df.drop_duplicates()

# Нижний регистр для образования и семейного положения
for column in ['education', 'family_status']:
    df[ column ] = df[ column ].str.lower()
    
# Зададим тип данных в колонках
df[ [ 'children', 'dob_years', 'total_income' ] ] = df[ [ 'children', 'dob_years', 'total_income' ] ].astype('int64')

# Посмотрим, что получилось
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21470 entries, 0 to 21524
Data columns (total 11 columns):
children            21470 non-null int64
dob_years           21470 non-null int64
education           21470 non-null object
education_id        21470 non-null int64
family_status       21470 non-null object
family_status_id    21470 non-null int64
gender              21470 non-null object
income_type         21470 non-null object
debt                21470 non-null int64
total_income        21470 non-null int64
purpose             21470 non-null object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


<div class="alert alert-danger">
<b>❌ Комментарий ревьюера:</b> Удалять дубли можно только после полной обработки данных:
        
- необходимо заполнить все пропуски
- обработать аномальные значения
- привести данные к единому регистру (избавиться от неявных дублей)
- после этого удалять явные дубли

<div class="alert alert-info"> <b>
Комментарий студента:
 
</b> Саша, привет. Давай на ты.

Мне кажется, ты не прочитала мои комменатарии выше на эту тему. :(

На этом этапе мы почиситили датасет -> регистры / знаки после запятой (особенно если ввод данных делали люди), могут помочь найти настоящие дубликаты.
    
"В новом задании предлагается сначала стандартизировать регистр и типы данных. Я предлагаю сначала удалить дубликаты, а потом уже приступить к стандартизации: во-первых, total_income имеет много знаков после запятой (может, значение было переведено из одной валюты в другую) - этим можно повысить "уникальность" строки; во-вторых, если предположить, что разный регистр в income_type - это следствие ручного ввода данных человеком, то это тоже поможет избежать удаления лишних строк."

</div>

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера V2:</b> Миша, еще раз прошу прощения! Вчера у нас были огромные проблемы с трекером, работы не прогружались, мы не могли их отправить. И твоего описания удаления дублей словно не было - видимо, они не прогрузились:( Сейчас я вижу это описание. 
    
Да, ты абсолютно прав. Но смотри: раздор по большей части вносит колонка education. Когда мы приведем ее к нижнему регистру, у нас вновь образуются явные дубли - и надо будет еще раз применить метод .drop_duplicates - таким образом, мы найдем еще 17 явных дублей:)

### 4 - Словари (lookup tables)

В датасете у нас есть две колонки, которые могут быть использованы как ключи - `education_id` и `family_status_id`. Выделим их и соответсвтующие словари в отдельные таблицы.

In [13]:
# Словарь education
lut_education = df[ [ 'education_id', 'education' ] ].drop_duplicates().reset_index( drop=True )

# Словарь family_status
lut_family_status = df[ [ 'family_status_id', 'family_status' ] ].drop_duplicates().reset_index( drop=True )

### 5 - Классификация `total_income`

На основании диапазонов, указанных ниже, создадим столбец `total_income_category` с категориями:
- **E**: 0 – 30,000
- **D**: 30,001 – 50,000
- **C**: 50,001 – 200,000
- **B**: 200,001 – 1,000,000
- **A**: 1,000,001 и выше.

In [29]:
# Функция для категоризации
def fun_categorizer_total_income( value_for_categorization ):
    """
    This function allocates a category for a variable based on its value. 

    Args:
        value_for_categorization ([float or int]): value for categorization

    Returns:
        - 'E' if value_for_categorization <= 30,000
        - 'D' if value_for_categorization = [30,001 : 50,000]
        - 'C' if value_for_categorization = [50,001 : 200,000]
        - 'B' if value_for_categorization = [200,001 : 1,000,000]
        - 'A' if value_for_categorization >= 1,000,001
    """
    if value_for_categorization <= 30e3:
        return 'E'
    elif 30e3 < value_for_categorization <= 50e3:
        return 'D'
    elif 50e3 < value_for_categorization <= 200e3:
        return 'C'
    elif 200e3 < value_for_categorization <= 1e6:
        return 'B'
    else:
        return 'A'

# Применим функцию к датасету
df[ 'total_income_category' ] = df.total_income.apply( fun_categorizer_total_income )

### 6 - Классификация `purpose`

Наконец, пора добавить стандартные категории к колонке `purpose`.

In [34]:
# Заколхозим
def fun_categorizer_purpose( string_for_categorization ):
    """
    This function categorizes a string if a particular set of characters is present. 

    Args:
        string_for_categorization [str]: string, where a set of characters is checked

    Returns:
        - 'операции с автомобилем' if string contains 'авто'
        - 'операции с недвижимостью' if string contains 'нед' or 'жил'
        - 'проведение свадьбы' if string contains 'свад'
        - 'получение образования' if string contains 'образ'
    """
    if 'авто' in string_for_categorization:
        return 'операции с автомобилем'
    elif 'свад' in string_for_categorization:
        return 'проведение свадьбы'
    elif 'образ' in string_for_categorization:
        return 'получение образования'
    else:
        return 'операции с недвижимостью'

df[ 'purpose_category' ] = df.purpose.apply( fun_categorizer_purpose )

На этом можно закончить предобработку данных. Пора заняться исследованием!

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера:</b>  Категоризация поможет ответить на вопросы ниже. 

<div class="alert alert-info"> <b>Комментарий студента:</b> 
     
Саша, я добавил категоризацию по `purpose`. См функцию в 2.6
    
</div>

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера V3:</b> Добавление корректно!

## Проверка гипотез

Нам надо проверить 4 гипотезы:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

Процесс проверки будет простым: для каждого из этих вопросов построим сводную таблицу. Строки таблицы будут соответствовать категориям, а столбцы - общему количеству людей в категории, количеству людей вернувших кредит в срок, количеству людей невернувших кредит в срок. На основе этих колонок вычислим соответсвтующие доли и потом сделаем вывод.

Скажем, если разница между категориями более 3% - это статистически значимо, а значит зависимость есть. В обртном случае - зависимости нет. Я не имею опыта в статистике, но подозреваю, что в серьезном упражнении 3% были бы более аргументированы.

<div class="alert alert-danger">
<b>❌ Комментарий ревьюера:</b> По заданию нам нужно проверить еще одну гипотезу: давай категоризируем данные для нее и ответим на вопрос

<div class="alert alert-info"> <b>Комментарий студента:</b> 
     
Да, но нам убрали тему лемматизации из проекта. Я даже не могу найти теперь этот урок в практических занятиях.

Без лемматизации эту категоризацию придется делать в полуручном режиме - нет?
    
</div>

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера V2:</b> Увы:( Можно сделать чуть элегантнее: сгруппировать данные по целям кредита, посчитать уникальные идентификаторы и отсортировать в порядке убывания. Так мы увидим основные категории кредита, по которым и категоризируем данные 
    

In [16]:
# Сделаем функцию, которая будет выдавать нам результаты
def fun_counter( data, rows ):
    # Сводная таблица #1 - влияние наличия детей
    pivot = pd.pivot_table( df, 
                            values='gender', 
                            index=rows, 
                            columns='debt', 
                            aggfunc='count', 
                            fill_value=0 )

    # Поправим названия колонок, чтобы легче было жить
    pivot = pivot.rename( columns={ 0:'people_without_debt_no', 1:'people_with_debt_no' }, level=0 )

    # Суммарное количество людей в категории
    pivot['people_total'] = pivot.people_without_debt_no + pivot.people_with_debt_no

    # Доля людей имеющих и не имеющих задолжность
    pivot['people_without_debt_per'] = pivot.people_without_debt_no / pivot.people_total
    pivot['people_with_debt_per'] = pivot.people_with_debt_no / pivot.people_total
      
    # Возвращаем табличку
    return pivot.sort_values( by=[ 'people_with_debt_per', 'people_total' ] )

### 1 - Зависимость между наличием детей и возвратом кредита в срок

Проверим, что получилось для первой гипотезы.

In [17]:
display( fun_counter( df, 'children' ) )

debt,people_without_debt_no,people_with_debt_no,people_total,people_without_debt_per,people_with_debt_per
children,,,,,
5,9,0,9,1.000000,0.000000
0,13060,1063,14123,0.924733,0.075267
3,303,27,330,0.918182,0.081818
1,4462,453,4915,0.907833,0.092167
2,1858,194,2052,0.905458,0.094542
4,37,4,41,0.902439,0.097561


Первое, что бросается в глаза - не все категории достаточно большие, чтобы делать выводы. Так, у нас есть мало людей у кого есть 3, 4 и 5 детей, но сильно больше (почти в 10 раз) людей, у кого нет детей, есть 1 или 2 ребенка.

**Вывод:** Тем не менее, соагласно нашему критерию в 3% непохоже, чтобы была какая-то значимая зависимость между наличием детей и возвратом кредита в срок. Во всех категориях вероятность возврата состовляет около 90%

### 2 - Зависимость между семейным положением и возвратом кредита в срок

Проверим, что получилось для второй гипотезы.

In [18]:
display( fun_counter( df, 'family_status' ) )

debt,people_without_debt_no,people_with_debt_no,people_total,people_without_debt_per,people_with_debt_per
family_status,,,,,
вдовец / вдова,896,63,959,0.934307,0.065693
в разводе,1110,85,1195,0.928870,0.071130
женат / замужем,11413,931,12344,0.924579,0.075421
гражданский брак,3774,388,4162,0.906776,0.093224
не женат / не замужем,2536,274,2810,0.902491,0.097509


В этом случае у нас более равномерная выборка (по сравнению с предыдущей гипотезой), поэтому посмотрим на все категории.

**Вывод:** В этом случае у нас чуть более выраженная зависимость. Гражданский брак и отсутствие жены / мужа явно снижает вероятность возврата кредита (хотя она все еще около 90%) по сравнению с остальными категориями (вероятность возврата около 93%). Интересно, что самая надежная категория - это вдовец / вдова, хотя, вероятно, это следствие относительно маленькой выборки.

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера:</b>  Верно

### 3 - Зависимость между уровнем дохода и возвратом кредита в срок

Проверим, что получилось для третьей гипотезы.

In [19]:
display( fun_counter( df, 'total_income_category' ) )

debt,people_without_debt_no,people_with_debt_no,people_total,people_without_debt_per,people_with_debt_per
total_income_category,,,,,
D,329,21,350,0.940000,0.060000
B,5158,386,5544,0.930375,0.069625
A,23,2,25,0.920000,0.080000
C,14199,1330,15529,0.914354,0.085646
E,20,2,22,0.909091,0.090909


Опять некоторые категгории получились очень маленькими - всего 25 людей в категории по доходу 'A' (что ожидаемо).

**Вывод:** В этом случае у нас тоже есть зависимость, хотя и не совсем, как я ожидал. Видно, что люди с наименьшим доходом имеют самую низкую вероятность возврата кредита (все еще около 90%). В то время как более высокий доход может значительно это вероятность повысить (до 93% в случае 'B'). Интересно, что первую строчку у нас все равно заняла категория 'D'.

### 4 - Зависимость между целями кредита и возвратом кредита в срок

Проверим, что получилось для последней гипотезы.

In [47]:
display( fun_counter( df, 'purpose_category' ) )

debt,people_without_debt_no,people_with_debt_no,people_total,people_without_debt_per,people_with_debt_per
purpose_category,,,,,
операции с недвижимостью,10031,782,10813,0.927680,0.072320
проведение свадьбы,2149,186,2335,0.920343,0.079657
получение образования,3644,370,4014,0.907823,0.092177
операции с автомобилем,3905,403,4308,0.906453,0.093547


**Вывод:** А вот это неожиданный результат. Похоже, что кредиты на операции с недвижимостью и на проведение свадьбы возвращают реже, чем на получение образования и операции с автомобилем. Хотя разица оказалось не очень большой (всего ~2%), она заметна. Еще интересно, как каждые из двух категорий сгруппировались друг с другом.

<div class="alert alert-info"> <b>Комментарий студента:</b> 
     
Саша, вот итоги исследования по целям кредита. Вывод внизу обновил
    
</div>

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера V3:</b> Отлично!

## Выводы

В этом проекте я прошел весь цикл анализа данных: от загрузки до проверки гипотез. В первой части, я вычистил данные: привел к стандартному виду, разобрался с пустыми значениями, убрал дубликаты, сделал разные категории. Это была самая трудозатратная часть. Во второй части, я провел относительно простой анализ и сравнил вероятность возвращения кредита в срок для разных срезов в данных.

В итоге, мы получили, что количество детей слабо влияет на вероятность возврата кредита (хотя есть небольшая зависимость - чем больше детей, тем вероятность возврата ниже). Вероятно, это связано с дополнительными расходами на содержание ребенка.

Семейный статус оказывает заметное влияние: так люди, состоящие в зарегистрированных отношениях более чаще возвращают кредиты. Вероятно, это связно с некоторым "взрослением" которое сопровождает решение вступить в брак.

Уровень дохода также может оказать влияние на вероятность возврата кредита: как можно ожидать, чем меньше доход - тем (примерно) вероятность меньше. Исключение составляет категория 'D', которая оказалась самой надежной. Возможно, здесь бы следовало пересмотреть группировку по доходу: как-то неравномерно у нас распределились заявители.

Наконец, цели кредита тоже влияют на вероятность возврата. Удивительно, что кредит на свадьбу возвращают чаще, чем кредит на образование. Возможно, это также связано с тем, что в свадьбе (или приобретении жилья) могут участвовать два человека, тогда как кредиты на образование (или покупку недвижимости) берутся в одиночестве...

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера:</b> Спасибо за работу! Проект получился очень хорошим. Давай сделаем его сильнее:
        
- удалим дубли после полной обработки данных
- ответим на четвертый вопрос, а именно как категории кредита влияют на его возврат

Оставила направляющие комментарии. Возвращаю тебе работу, чтобы ты смог задать вопросы. Буду ждать твою работу на финальное ревью!
</div>

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера V2:</b> Спасибо за проработку комментариев. Еще раз прошу прощения за проблемы с треккером. Давай дорабоатем ответ на четвертый вопрос, после чего проект будет принят. Буду ждать твою работу!

<div class="alert alert-info"> <b>Комментарий студента:</b> 
     
Саша, добавил анализ по последнему вопросу. Дубликаты не менял (но там буквально строчку кода перенсти в конец и все заработает, если прям очень хочется :) )
    
Дай знать, если надо будет что-то еще поправить. Спасибо, что занимаешься с нами <3
    
</div>

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера V3:</b> Миша, спасибо за тщательную работу над проектом и проработку комментариев! Теперь все отлично. Если у тебя возникнут вопросы, смело задавай мне их через куратора. Удачи на следующих этапах!